In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
states_map = {
    'AZ':'Arizona',
    'CT':'Connecticut',
    'DE':'Delaware',
    'FL':'Florida',
    'HI':'Hawaii',
    'IN':'Indiana',
    'MA':'Massachusetts',
    'MD':'Maryland',
    'ME':'Maine',
    'MI':'Michigan',
    'MN':'Minnesota',
    'MT':'Montana',
    'ND':'North Dakota',
    'NE':'Nebraska',
    'NJ':'New Jersey',
    'NM':'New Mexico',
    'NV':'Nevada',
    'NY':'New York',
    'OH':'Ohio',
    'PA':'Pennsylvania',
    'RI':'Rhode Island',
    'TN':'Tennessee',
    'TX':'Texas',
    'UT':'Utah',
    'VA':'Virginia',
    'VT':'Vermont',
    'WA':'Washington',
    'WI':'Wisconsin',
    'WV':'West Virginia',
    'WY':'Wyoming'
}

In [3]:
demographics = pd.read_csv('demographics_train.csv')
election = pd.read_csv('election_train.csv')

## TASK 1

In [4]:
election = election.pivot_table(
    index = ['State','County','Office','Year']
    ,columns = 'Party'
    ,values= 'Votes'
).reset_index()

In [5]:
election[election["Democratic"].isna()]

Party,State,County,Office,Year,Democratic,Republican
425,NE,Lancaster County,US Senator,2018,NaN,NaN
716,TN,Meigs County,US Senator,2018,NaN,NaN
753,TX,Bee County,US Senator,2018,NaN,NaN
869,TX,Menard County,US Senator,2018,NaN,NaN
1119,WI,Lafayette County,US Senator,2018,NaN,NaN


## TASK 2

In [6]:
election.head(1)

Party,State,County,Office,Year,Democratic,Republican
0,AZ,Apache County,US Senator,2018,16298.0,7810.0


In [7]:
#map the state initials to the actual full state name
election['State'] = election['State'].map(states_map) #DO THIS ONLY ONCE
election['State'] = election['State'].str.lower()

#get all the indexes where the county doesnt have 'county' in the name already 
no_county_name = election[election["County"].str.contains("County") == False]
add_county = [no_county_name.index.values]

#add 'county' into those that are missing it
for index_val in add_county:
    election.loc[index_val,'County'] = election.loc[index_val,'County'] + ' County'
election.County = election.County.str.lower()

    
election.head(3)

Party,State,County,Office,Year,Democratic,Republican
0,arizona,apache county,US Senator,2018,16298.0,7810.0
1,arizona,cochise county,US Senator,2018,17383.0,26929.0
2,arizona,coconino county,US Senator,2018,34240.0,19249.0


In [8]:
#check if County has any that already contain the word 'county' / none do
demographics[demographics["County"].str.contains("County") == True] 
demographics['State'] = demographics['State'].str.lower()

demographics.County = demographics.County + ' County' #DO THIS ONLY ONCE 
demographics.County = demographics.County.str.lower()

demographics.head(3)

,State,County,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,wisconsin,la crosse county,55063,117538,0,90.537528,1.214075,1.724549,2.976059,51.171536,43.241335,14.702479,51477,4.796952,5.474767,67.529757,16.827753
1,virginia,alleghany county,51005,15919,12705,91.940449,5.207614,1.432251,1.300333,51.077329,31.660280,23.902255,45538,4.560986,15.537543,83.711604,52.393846
2,indiana,fountain county,18045,16741,12750,95.705155,0.400215,2.359477,1.547100,49.770026,35.899887,18.941521,45924,7.978789,12.032155,85.538940,65.951276


In [9]:
merged_df = pd.merge(
    demographics, election, how = 'inner', 
    on = ['State','County']
)


In [10]:
len(merged_df) #we have 1200 rows which aligns with the hint

1200

## TASK 3

In [11]:
len(merged_df.columns) #21 variables
merged_df.dtypes

State                                     object
County                                    object
FIPS                                       int64
Total Population                           int64
Citizen Voting-Age Population              int64
Percent White, not Hispanic or Latino    float64
Percent Black, not Hispanic or Latino    float64
Percent Hispanic or Latino               float64
Percent Foreign Born                     float64
Percent Female                           float64
Percent Age 29 and Under                 float64
Percent Age 65 and Older                 float64
Median Household Income                    int64
Percent Unemployed                       float64
Percent Less than High School Degree     float64
Percent Less than Bachelor's Degree      float64
Percent Rural                            float64
Office                                    object
Year                                       int64
Democratic                               float64
Republican          

> FIPS - Take out variable - The FIPS (federal information processing standard) is not necessary. This is a code that helps identify locations, but since we already have the state and county and have votes based on these two regional identifiers, we don't need the FIPS. 

> Total Population - Take out variable - The total population will not have any effect on the number of people that are going to vote. The voting-age population is more representative of who has the ability to vote.

> Year and Office - Take out variable - both of these are all the same for each variable and so hold no value

In [12]:
new_df = merged_df.copy(deep=True) #copy the dataset so that any modifications on new_df, 
                                   #no changes happen to merged_df in case we need to backtrack

In [13]:
new_df = new_df.drop(["FIPS","Year","Office"], axis=1) #drop vars

In [14]:
new_df.head(1)

,State,County,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Democratic,Republican
0,wisconsin,la crosse county,117538,0,90.537528,1.214075,1.724549,2.976059,51.171536,43.241335,14.702479,51477,4.796952,5.474767,67.529757,16.827753,35731.0,21160.0


## TASK 4

In [15]:
print("Max rows", len(new_df)) #this shows the total number of rows

#this will give us the columns with zeroes and the num of zeroes
cols_zeroes = pd.DataFrame(columns = ["Column","Rows with zero"])

index = 0
for column in new_df:
    count = len(new_df[new_df[column] == 0])
    index +=1
    cols_zeroes.loc[index] = [column,count]

print("\nColumns with zero values\n", cols_zeroes)
#we can see that the citizen voting-age population has 680/1200 rows missing data
#i think it is better to drop the column rather than the rows. let me know what you think

new_df.drop("Citizen Voting-Age Population", axis=1,inplace=True)

#we also have other zero values, however...
new_df[new_df["Percent Black, not Hispanic or Latino"] == 0]
#the majority populations are very small in nature, and are predominantly white
#it is not outlandish to reason that there may be a population of zero black.
#this is similar reasoning pushed onto the remaining zero values, where it is 
#not impossible to have values of zero.

#____________________________________________________________________________________
#this will show us which columns have NA values
has_na_vals = []
for column in new_df:
    check_na = new_df[new_df[column].isna()]
    if check_na.empty == False:
        has_na_vals.append([column,len(check_na)])

print("\nColumns with NA values\n", has_na_vals)

#we have 5 rows of dem/rep row missing

#this shows the missing rows are both in the same columns. it is a small subset, so opt to take those out
temp_df = new_df[new_df["Republican"].isna()]
na_rows = temp_df.index

new_df.drop(na_rows, axis=0, inplace=True)

#confirms we dropped it, we now have 1195 rows
len(new_df)

Max rows 1200

Columns with zero values
                                    Column Rows with zero
1                                   State              0
2                                  County              0
3                        Total Population              0
4           Citizen Voting-Age Population            680
5   Percent White, not Hispanic or Latino              0
6   Percent Black, not Hispanic or Latino             45
7              Percent Hispanic or Latino              5
8                    Percent Foreign Born              3
9                          Percent Female              0
10               Percent Age 29 and Under              0
11               Percent Age 65 and Older              0
12                Median Household Income              0
13                     Percent Unemployed              3
14   Percent Less than High School Degree              0
15    Percent Less than Bachelor's Degree              0
16                          Percent Rural      

1195

## TASK 5

In [16]:
new_df["Party"] = np.where(new_df["Democratic"] > new_df["Republican"], 1,0)
new_df.head(1)

,State,County,Total Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural,Democratic,Republican,Party
0,wisconsin,la crosse county,117538,90.537528,1.214075,1.724549,2.976059,51.171536,43.241335,14.702479,51477,4.796952,5.474767,67.529757,16.827753,35731.0,21160.0,1


## TASK 6

In [17]:
dem_pop = new_df[new_df['Party'] == 1]['Total Population'].mean()
rep_pop = new_df[new_df['Party'] == 0]['Total Population'].mean()

dem_std = new_df[new_df['Party'] == 1]['Total Population'].std()
rep_std = new_df[new_df['Party'] == 0]['Total Population'].std() #wide var difference

print("dem mean: ", dem_pop,"\nrep mean: ", rep_pop) #democratic population is higher
print("dem std: ",dem_std,"\nrep std: ", rep_std,"\n")

#null = population difference is within reasonable bounds
#alternate = population difference is statistically significant

dem_pop_array = new_df[new_df['Party'] == 1]['Total Population']
rep_pop_array = new_df[new_df['Party'] == 0]['Total Population']

[statistic, pvalue] = st.ttest_ind(dem_pop_array, rep_pop_array, equal_var = False)
print(statistic)
print(pvalue/2) #is e^-14, so it is a very small number

#based on this, this is statistically significant. we reject the null hypothesis based on pval

dem mean:  300998.3169230769 
rep mean:  53864.6724137931
dem std:  553600.0257123302 
rep std:  94192.57279397613 

8.004638577960957
1.0239358801486512e-14


## TASK 7

In [18]:
dem_med = new_df[new_df['Party'] == 1]['Median Household Income'].mean()
rep_med = new_df[new_df['Party'] == 0]['Median Household Income'].mean()

dem_std = new_df[new_df['Party'] == 1]['Median Household Income'].std()
rep_std = new_df[new_df['Party'] == 0]['Median Household Income'].std() #wide var difference

print("dem mean: ", dem_med,"\nrep mean: ", rep_med) #democratic population is higher
print("dem std: ",dem_std,"\nrep std: ", rep_std,"\n")

#null = population income is within reasonable bounds
#alternate = population income is statistically significant

dem_med_array = new_df[new_df['Party'] == 1]['Median Household Income']
rep_med_array = new_df[new_df['Party'] == 0]['Median Household Income']

[statistic, pvalue] = st.ttest_ind(dem_med_array, rep_med_array, equal_var = False)
print(statistic)
print(pvalue/2) #is e^-08, so it is a very small number

#based on this, this is statistically significant. we reject the null hypothesis based on pval

dem mean:  53798.732307692306 
rep mean:  48746.81954022989
dem std:  15289.130077404618 
rep std:  10670.72941182022 

5.479141589767388
3.574718681591286e-08


In [22]:
# new_df
import plotly